# Aprendizagem por regras

$\quad$ Basicamente a partir de uma base de dados o algoritmo irá gerar regras para classificação dos dados. Inicialmente usaremos o algoritmo _OneR_, o qual irá criar apenas uma regra.

$\quad$ Comparando os algoritmos de aprenzagem por regras e árvore de decisões vemos que os dois tem a vantagem de compreensibilidade, é muito fácil entender o resultado do algoritmo. Os dois precisam de pouco espaço de armazenamento também. No entanto, o algoritmo de regras é mais lento que o de árvores. Além disso, algoritmo de regras não apresentam melhores resultados do que árvores de decisão.

## 1. Base risco crédito

In [1]:
import Orange

In [2]:
base_risco_credito = Orange.data.Table('risco_credito_regras.csv')
base_risco_credito

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

In [3]:
base_risco_credito.domain

[historia, divida, garantias, renda | risco]

### 1.1 Algoritmo CN2

In [4]:
cn2 = Orange.classification.rules.CN2Learner()
regras_risco_credito = cn2(base_risco_credito)

In [5]:
for regras in regras_risco_credito.rule_list:
    print(regras)

IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=baixo 
IF TRUE THEN risco=alto 


$\quad$ O algoritmo acima criou regras para classificação de registros de crédito.

$\quad$ Agora faremos os teste dessas regras.

In [6]:
# história boa, dívida alta, garantias nenhuma, renda > 35
#história ruim, dívida alta, garantias adequada, renda < 15
previsoes = regras_risco_credito([['boa', 'alta', 'nenhuma', 'acima_35'], ['ruim', 'alta', 'adequada', '0_15']])
previsoes

array([1, 0], dtype=int64)

In [7]:
base_risco_credito.domain.class_var.values
#vemos que a classificação 0 significa risco alto, a classificação 1 risco baixo e o moderado é 2

('alto', 'baixo', 'moderado')

In [8]:
for i in previsoes:
    print(base_risco_credito.domain.class_var.values[i])

baixo
alto


## 2. Base credit data

In [9]:
base_credit = Orange.data.Table('credit_data_regras.csv')

In [10]:
base_credit.domain

[income, age, loan | default]

In [11]:
base_dividida = Orange.evaluation.testing.sample(base_credit, n = 0.25)
base_dividida

([[43940.9, 38.306, 2855.38 | 0],
  [34667, 18.8532, 2827.29 | 0],
  [33265.8, 21.5835, 5968.44 | 1],
  [35950.5, 35.4718, 2664.93 | 0],
  [49352.3, 57.5422, 3362.77 | 0],
  ...
 ],
 [[36029.3, 52.6406, 2928.1 | 0],
  [55066.2, 55.7392, 9332.7 | 0],
  [43044.5, 60.8484, 1661.71 | 0],
  [33274.1, 23.9534, 2244.88 | 0],
  [61398.7, 63.2944, 9008.15 | 0],
  ...
 ])

In [12]:
base_dividida[0]

[[43940.9, 38.306, 2855.38 | 0],
 [34667, 18.8532, 2827.29 | 0],
 [33265.8, 21.5835, 5968.44 | 1],
 [35950.5, 35.4718, 2664.93 | 0],
 [49352.3, 57.5422, 3362.77 | 0],
 ...
]

In [13]:
base_treinamento = base_dividida[1]
base_teste = base_dividida[0]

In [14]:
len(base_treinamento), len(base_teste)

(1500, 500)

In [15]:
cn2 = Orange.classification.rules.CN2Learner()
regras_credit = cn2(base_treinamento)

In [16]:
for regras in regras_credit.rule_list:
    print(regras)

IF age>=34.9966813726351 THEN default=0 
IF loan<=2507.64970973955 AND income>=20145.9885970689 THEN default=0 
IF loan>=7660.34617051509 AND loan>=9601.375482171099 THEN default=1 
IF income<=31702.3342987522 AND age>=32.4554232852533 THEN default=1 
IF age>=33.9493660898879 AND income>=44405.28066363191 THEN default=0 
IF age<=18.075335860718 AND income>=63287.038907874405 THEN default=0 
IF loan>=7660.34617051509 AND age>=20.9909665295854 THEN default=1 
IF loan>=5898.0008931423 AND loan>=8402.415585756991 THEN default=1 
IF income>=58132.4712652713 THEN default=0 
IF loan>=5883.660557890999 AND age>=29.4230185477063 THEN default=1 
IF loan>=6361.97343810153 THEN default=1 
IF income>=45311.831838917 AND income>=50360.6787929311 THEN default=0 
IF income<=31702.3342987522 AND age>=31.9685264034197 THEN default=0 
IF income<=31702.3342987522 AND age>=29.297398745548 THEN default=1 
IF income<=31702.3342987522 AND age>=29.2350565218367 THEN default=0 
IF income<=31702.3342987522 AND l

In [17]:
previsoes = Orange.evaluation.testing.TestOnTestData(base_treinamento, base_teste, [lambda testdata: regras_credit])

In [18]:
previsoes

In [19]:
Orange.evaluation.CA(previsoes)

array([0.978])

ou seja, a precisão do algoritmodoi de 97.6%, mas vemos que o resultado foir pior que o da árvore de decisão

## 3. Classificador base - Majority learner

$\quad$ Usar Majority learner é um ótimo algoritmo para se ter uma precisão base. Seus resultados podem ser utilizados para comparar com outros algoritmos, visnado observar se são boas opções para serem utilizadas. Esse algoritmo define o percentual de acerto a partir da maioria dos registros.

### 3.1 Base Credit Data

In [20]:
base_credit = Orange.data.Table('credit_data_regras.csv')

In [21]:
majority = Orange.classification.MajorityLearner()

In [22]:
previsoes = Orange.evaluation.testing.TestOnTestData(base_credit, base_credit, [majority])

In [23]:
Orange.evaluation.CA(previsoes)

array([0.8585])

In [29]:
for registro in base_credit:
    print(registro.get_class())

0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
1
1
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
1
0
0
1
0
0
0
0


In [31]:
from collections import Counter
Counter(str(registro.get_class()) for registro in base_credit)

Counter({'0': 1717, '1': 283})

In [32]:
1717/2000

0.8585

a precisão desse algoritmo é de 85.85%. Essa precisão serve como uma precisão base, caso um algoritmo tenha uma precisão menor que essa, significa que não vale a pena usá-lo, caso contrário vale a pena usar o algoritmo.

### 3.2 Base Census

In [37]:
base_census = Orange.data.Table('census_regras.csv')
base_census.domain

[age, workclass, final-weight, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loos, hour-per-week, native-country | income]

In [39]:
majority = Orange.classification.MajorityLearner()
previsoes = Orange.evaluation.testing.TestOnTestData(base_census, base_census, [majority])
Orange.evaluation.CA(previsoes)

array([0.75919044])

In [40]:
Counter(str(registro.get_class()) for registro in base_census)

Counter({'<=50K': 24720, '>50K': 7841})

In [41]:
24720/(24720 + 7841)

0.7591904425539756